In [15]:
import pandas as pd
import numpy as np
from scipy import stats
from theano import shared
import pymc3 as pm
import pickle
import sys
print(sys.version) 

"""
This notebook allows to run the GP simulation for sub pixel spatial variability of tundra snow
with the SMRT model. The Gaussian processes implementation is done with pymc3 and the radiative transfer is SMRT.

This code has to be run before Bias_simulation_mean_GP.ipynb and provide inputs for it. The next notebook
allows for bias and RMSE calculation to be done with measurement from the SSMIS satellite sensor
"""


3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]


'\nThis notebook allows to run the GP simulation for sub pixel spatial variability of tundra snow\nwith the SMRT model. The Gaussian processes implementation is done with pymc3 and the radiative transfer is SMRT.\n\nThis code has to be run before Bias_simulation_mean_GP.ipynb and provide inputs for it. The next notebook\nallows for bias and RMSE calculation to be done with measurement from the SSMIS satellite sensor\n'

In [20]:
import scipy
print(np.__version__)
print(pd.__version__)
print(pm.__version__)
scipy.__version__

1.16.4
0.25.1
3.9.3


'1.3.1'

In [9]:
#import data of layer ratio
#Cambay
filename = 'ratio_WS_DH_snowpit2015.csv'
df_cb15 = pd.read_csv(filename)
filename = 'ratio_WS_DH_snowpit2016.csv'
df_cb16 = pd.read_csv(filename)
filename = 'ratio_WS_DH_snowpit2017.csv'
df_cb17 = pd.read_csv(filename)
filename = 'ratio_WS_DH_snowpit2018_NDVI_Sx.csv'
df_cb18 = pd.read_csv(filename)
filename = 'ratio_WS_DH_snowpit2019_NDVI_Sx.csv'
df_cb19 = pd.read_csv(filename)

# TVC data
filename = 'ratio_layer_TVC_March2018_strat_update_NDVI_TPI.csv'
df_tvc18 = pd.read_csv(filename)
filename = 'ratio_layer_TVC_March2019_strat_update_NDVI_TPI.csv'
df_tvc19 = pd.read_csv(filename)

#Creating dataframe input for Gaussian process
X = df_cb15['thickness'].append(df_cb16['thickness']).append(df_cb17['thickness']).append(df_cb18['thickness']).append(df_cb19['thickness']).append(df_tvc18['total_depth (cm)']).append(df_tvc19['total_depth (cm)'])/100
Y_DH = df_cb15['% DH'].append(df_cb16['% DH']).append(df_cb17['% DH']).append(df_cb18['% DH']).append(df_cb19['% DH']).append(df_tvc18['DH']).append(df_tvc19['DH'])
Y_WS = df_cb15['% WS'].append(df_cb16['% WS']).append(df_cb17['% WS']).append(df_cb18['% WS']).append(df_cb19['% WS']).append(df_tvc18['WS']).append(df_tvc19['WS'])

#removing NaN
df = pd.DataFrame({'x' : X, 'y_DH' : Y_DH, 'y_WS' : Y_WS})
df = df.drop(df[np.isnan(df.y_DH) == True ].index)
df.y_DH[df.y_DH == 0] =0.01
df.y_DH[df.y_DH == 1] =0.99



In [10]:
#Building the Gaussian process fit with pymc3
#theano is already install in pymc3

# Defining the invLogit Mean Function
import theano.tensor as tt

class InvLogit(pm.gp.mean.Mean):
    """
    InvLogit function for Gaussian process. Customn Mean function
    """

    def __init__(self, a, x0, c, d):
        pm.gp.mean.Mean.__init__(self)
        self.a = a
        self.x0 = x0
        self.c = c
        self.d = d

    def __call__(self, X):
        return (self.d * pm.math.invlogit(self.a*(X - self.x0)) + self.c).reshape((-1,))

"""
begin GP Model
"""
#train model
with pm.Model() as gp_model_1:

    #param from figure 5 in paper 
    #custom Mean function
    mean_func = InvLogit(a = np.array([-5]), x0 = np.array([0.5]), c = np.array([0.2]), d = np.array([0.35]))
    
    def logistic(x, a, x0, c, d):
    # a is the slope, x0 is the location
        return d * pm.math.invlogit(a*(x - x0)) + c
    
    #param from figure 5 in paper
    #params for invlogit for Cov func
    a = -5.0
    x0 = 0.6
    c = 0.25
    d = 1.5
    sigma = pm.HalfNormal('sigma', sd =0.1)
    cov_base = pm.gp.cov.WhiteNoise(0.15)
    cov_func = pm.gp.cov.ScaledCov(1, scaling_func=logistic, args=(a,x0,c,d), cov_func=cov_base)

    # Specify the GP
    gp = pm.gp.Latent(mean_func = mean_func, cov_func=cov_func)
    # make gp prior
    f = gp.prior("f", X=df.x.values.reshape(-1,1))
    p = pm.Deterministic("p", pm.math.invlogit(f))
    y = pm.Beta("y", mu = p, sigma = sigma, observed = df.y_DH.values)
    #mp = pm.find_MAP()
    gp_trace1 = pm.sample(1000, tune=1000, chains=2)
    
#set up parameter log-normal on new X with shared variables
zeta = np.sqrt(np.log(1 + 0.4**2))
lam = np.log(0.4) - (zeta**2)/2
n_sub = 500
#set up conditional and shared variable for the prediction
with gp_model_1:
    x_depth = stats.lognorm.rvs(scale = np.exp(lam),s = zeta, loc = 0, size = n_sub).reshape(-1,1)
    x_tensor = shared(value = x_depth)
    f_pred = gp.conditional(f'y_pred', x_tensor, shape = n_sub)

    

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [f_rotated_, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 22 seconds.
/home/julien/anaconda3/envs/bayes/lib/python3.7/site-packages/pymc3/gp/cov.py:93: UserWarning: Only 1 column(s) out of Subtensor{int64}.0 are being used to compute the covariance function. If this is not intended, increase 'input_dim' parameter to the number of columns to use. Ignore otherwise.
  " the number of columns to use. Ignore otherwise.", UserWarning)


In [11]:
##check trace if necessary once, leave as comment otherwise
#pm.traceplot(gp_trace1)

AttributeError: module 'smrt' has no attribute '__version__'

In [12]:
from smrt import make_soil, sensor_list, make_model, make_snowpack, make_interface
from smrt.inputs.make_medium import make_generic_stack
from smrt.core.globalconstants import DENSITY_OF_ICE
from smrt.atmosphere.simple_isotropic_atmosphere import SimpleIsotropicAtmosphere

def ssa_to_l(ssa, density):
    #converting SSA to correlation length
    f = density/DENSITY_OF_ICE
    l_d = 4 * (1-f)/(DENSITY_OF_ICE*ssa)
    return l_d


def simu_mean(mu, temp_surface, temp_base):
    n_sub = 500
    #mu =0.34
    CV = list(np.linspace(0.1, 1, 40))
    #mean ratio of both site
    ratio_dh = 0.46
    ratio_ws = 0.54
    #mean density from table 5
    density_DH = 266
    density_WS = 335

    #correlation length calculation with kapp = 1.39
    #SSA for DH 11 (table 5)
    #SSA for WS 20 (table 5)
    l_DH = 1.39 * ssa_to_l(11, density_DH)
    l_WS = 1.39 * ssa_to_l(20, density_WS)

    #Tb for mean depth
    #37
    sp = make_snowpack(thickness= [mu* ratio_dh, mu*ratio_ws],
                   microstructure_model = 'exponential',
                   density = [density_WS, density_DH],
                   corr_length = [l_WS, l_DH],                 
                   temperature = [temp_surface, temp_base])
    substrate = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = temp_base, roughness_rms = 0.017)
    medium = sp + substrate
    radiometer = sensor_list.passive(37e9, 55)
    model = make_model("iba", 'dort')
    res = model.run(radiometer, medium)
    print(f' 37 Mean depth TbV : {res.TbV()}, TbH : {res.TbH()}')
    
    return res.TbV(), res.TbH()

In [13]:
def simu_GP(mu, temp_surface, temp_base):
#Tb fo GP process and log Normal 
    n_sub = 500
    Tb_cv_37 = []
    
    CV = list(np.linspace(0.1, 1, 40))
    #mean ratio of both site
    ratio_dh = 0.46
    ratio_ws = 0.54
    #mean density from table 5
    density_DH = 266
    density_WS = 335
    #correlation length calculation with kapp = 1.39
    #SSA for DH 11 (table 5)
    #SSA for WS 20 (table 5)
    l_DH = 1.39 * ssa_to_l(11, density_DH)
    l_WS = 1.39 * ssa_to_l(20, density_WS)
    
    for cv in CV:
        #for error estimate, run 10 times
        #error_h, error_v = [], []
        #for n in range(0,10):   
        zeta = np.sqrt(np.log(1 + cv**2))
        lam = np.log(mu) - (zeta**2)/2
        swe_pred = []
        with gp_model_1:
            x_depth = stats.lognorm.rvs(scale = np.exp(lam),s = zeta, loc = 0, size = n_sub).reshape(-1,1)
            x_tensor.set_value(x_depth)
            y_samples = pm.sample_posterior_predictive(gp_trace1, vars=[f_pred], samples = 3, progressbar = False)

        DH = y_samples[f'y_pred'][1,:]
        DH = np.where(DH >= 0, DH, 0.01)
        DH = np.where(DH <= 1.0, DH, 0.99)

        radiometer = sensor_list.passive(37e9, 55)
        model = make_model("iba", 'dort')
        tbv, tbh = [], []
        for idx, d in np.ndenumerate(x_depth):
            thick = [d * DH[idx[0]], d * (1 - DH[idx[0]])]
            sp = make_snowpack(thickness= thick,
                           microstructure_model = 'exponential',
                           density = [density_WS, density_DH],
                           corr_length = [l_WS, l_DH],                 
                           temperature = [temp_surface, temp_base])
            substrate = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = temp_base, roughness_rms = 0.017)
            medium = sp + substrate
            res = model.run(radiometer, medium)
            tbv.append(res.TbV())
            tbh.append(res.TbH())

        #error_h.append(np.mean(tbh))
        #error_v.append(np.mean(tbv))

        mean_tbv, mean_tbh = np.mean(tbv), np.mean(tbh)
        #std_tbv, std_tbh = np.std(error_v), np.std(error_h)
        print(f'CV : {cv}, tbv : {mean_tbv} , tbh : {mean_tbh}')
        #print(f'CV : {cv}, tbv : {mean_tbv} ± {std_tbv}, tbh : {mean_tbh} ± {std_tbh}')
        Tb_cv_37.append({'cv' : cv, 'mean_tbv' : mean_tbv, 'mean_tbh' : mean_tbh, 
                         'tbv' : tbv, 'tbh' : tbh})
        
    return Tb_cv_37


## CB

In [14]:
#Tb for mean depth for CB
#temperature and mean depth from table 1 and table 2 
# mean depth = 0.42, temp 260, 257
TBV_cb_19, TBH_cb_19 = simu_mean(0.42, 261, 257)
# mean depth = 0.34, temp 260, 257
TBV_cb_18, TBH_cb_18 = simu_mean(0.34, 261, 257)
# mean depth = 0.42, temp 260, 257
TBV_cb_17, TBH_cb_17= simu_mean(0.42, 261, 257)
# mean depth = 0.28, temp 257, 257
TBV_cb_16, TBH_cb_16 = simu_mean(0.28, 255, 255)
# mean depth = 0.42, temp 256, 257
TBV_cb_15, TBH_cb_15 = simu_mean(0.32, 255, 253)

 37 Mean depth TbV : 186.15513203454074, TbH : 167.1910251328983
 37 Mean depth TbV : 190.39378129864025, TbH : 171.284260932607
 37 Mean depth TbV : 186.15513203454074, TbH : 167.1910251328983
 37 Mean depth TbV : 192.1336320252846, TbH : 173.20109132124063
 37 Mean depth TbV : 186.8463315727031, TbH : 168.22822803832068


In [15]:
#GP simulation for CB-42 cm depth
Tb_cv_cb = simu_GP(0.42, 261, 257)
file_tb_cv = open('results_tb_cv_sd42_CB.obj', 'wb')
pickle.dump(Tb_cv_cb, file_tb_cv)

/home/julien/anaconda3/envs/bayes/lib/python3.7/site-packages/pymc3/sampling.py:1708: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


CV : 0.1, tbv : 182.47175119221927 , tbh : 163.97226885193592


/home/julien/anaconda3/envs/bayes/lib/python3.7/site-packages/pymc3/sampling.py:1708: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


CV : 0.12307692307692308, tbv : 182.91368678430945 , tbh : 164.36261155134918
CV : 0.14615384615384616, tbv : 181.3299395361228 , tbh : 162.92729767414716
CV : 0.16923076923076924, tbv : 181.30436405626648 , tbh : 162.9038755077726
CV : 0.19230769230769232, tbv : 182.19513999009382 , tbh : 163.71424932613067
CV : 0.2153846153846154, tbv : 183.12155759725286 , tbh : 164.56506362162258
CV : 0.23846153846153847, tbv : 182.6724608736354 , tbh : 164.16543521346662
CV : 0.26153846153846155, tbv : 183.71316006124644 , tbh : 165.1326149384369
CV : 0.2846153846153846, tbv : 183.41307544160267 , tbh : 164.85425358944968
CV : 0.3076923076923077, tbv : 184.20139777583523 , tbh : 165.62808793215942
CV : 0.3307692307692308, tbv : 183.60145430981578 , tbh : 165.06879957994093
CV : 0.3538461538461538, tbv : 185.43607852148898 , tbh : 166.80721004204534
CV : 0.3769230769230769, tbv : 184.77980570351784 , tbh : 166.1482364890053
CV : 0.4, tbv : 184.22347707856574 , tbh : 165.65135528640496
CV : 0.423076

In [16]:
#GP simulation for CB-34-28-32 cm depth
Tb_cv_cb = simu_GP(0.32, 261, 257)
file_tb_cv = open('results_tb_cv_sd32_CB.obj', 'wb')
pickle.dump(Tb_cv_cb, file_tb_cv)

CV : 0.1, tbv : 191.7002813238821 , tbh : 172.76428227161384
CV : 0.12307692307692308, tbv : 189.9869002768815 , tbh : 171.15891221411871
CV : 0.14615384615384616, tbv : 190.9164257037989 , tbh : 172.04399857924722
CV : 0.16923076923076924, tbv : 190.96619816082818 , tbh : 172.07121109152504
CV : 0.19230769230769232, tbv : 191.87899402836098 , tbh : 172.9605411565767
CV : 0.2153846153846154, tbv : 191.65408557144818 , tbh : 172.7641750134616
CV : 0.23846153846153847, tbv : 193.67214985985916 , tbh : 174.6434294983601
CV : 0.26153846153846155, tbv : 190.58628560492343 , tbh : 171.74977160266323
CV : 0.2846153846153846, tbv : 192.71354724942483 , tbh : 173.76419670889166
CV : 0.3076923076923077, tbv : 192.08664705307507 , tbh : 173.15908366119936
CV : 0.3307692307692308, tbv : 192.83623103158482 , tbh : 173.91142885921275
CV : 0.3538461538461538, tbv : 193.82433705477294 , tbh : 174.85958773204456
CV : 0.3769230769230769, tbv : 193.2161921607917 , tbh : 174.26094662282256
CV : 0.4, tbv :

## TVC

In [17]:
#Tb for mean depth for tvc
#temperature and mean depth from table 1 and table 2 
#mean depth = 0.42, temp 260, 263
TBV_tvc_19, TBH_tvc_19 = simu_mean(0.42, 260, 263)
# mean depth = 0.39, temp 260, 263
TBV_tvc_18, TBH_tvc_18 = simu_mean(0.39, 260, 263)

 37 Mean depth TbV : 189.27815650368944, TbH : 170.0762860205117
 37 Mean depth TbV : 190.69459040855952, TbH : 171.44436258332786


In [18]:
Tb_cv_tvc = simu_GP(0.41, 260, 263)
file_tb_cv_37 = open('results_tb_cv_sd41_TVC.obj', 'wb')
pickle.dump(Tb_cv_tvc, file_tb_cv_37)

CV : 0.1, tbv : 187.05826828153278 , tbh : 168.19286842404358
CV : 0.12307692307692308, tbv : 185.27972399862372 , tbh : 166.54525455844228
CV : 0.14615384615384616, tbv : 186.69820324414587 , tbh : 167.88673642185535
CV : 0.16923076923076924, tbv : 187.23480584987956 , tbh : 168.36049675866147
CV : 0.19230769230769232, tbv : 185.89315175632595 , tbh : 167.0864363792953
CV : 0.2153846153846154, tbv : 186.18514155730114 , tbh : 167.40891552752322
CV : 0.23846153846153847, tbv : 187.4398667576725 , tbh : 168.5527471537949
CV : 0.26153846153846155, tbv : 186.12108438728083 , tbh : 167.34551983434383
CV : 0.2846153846153846, tbv : 188.7900920407131 , tbh : 169.8265033856568
CV : 0.3076923076923077, tbv : 187.74440852564084 , tbh : 168.895789871999
CV : 0.3307692307692308, tbv : 186.97020506126745 , tbh : 168.17232189079027
CV : 0.3538461538461538, tbv : 188.11501340840712 , tbh : 169.23350587027375
CV : 0.3769230769230769, tbv : 188.6198293484882 , tbh : 169.7115243771039
CV : 0.4, tbv : 1

### Put all simu mean in dict

In [19]:
dict_tbh = {'cb_19' : TBH_cb_19, 'cb_18' : TBH_cb_18, 'cb_17' : TBH_cb_17, 'cb_16' : TBH_cb_16,
            'cb_15' : TBH_cb_15, 'tvc_19' : TBH_tvc_19, 'tvc_18' : TBH_tvc_18}

dict_tbv = {'cb_19' : TBV_cb_19, 'cb_18' : TBV_cb_18, 'cb_17' : TBV_cb_17, 'cb_16' : TBV_cb_16,
            'cb_15' : TBV_cb_15, 'tvc_19' : TBV_tvc_19, 'tvc_18' : TBV_tvc_18}

fileh = open("Tbh_simu_mean.obj","wb")
pickle.dump(dict_tbh,fileh)

fileV = open("Tbv_simu_mean.obj","wb")
pickle.dump(dict_tbv,fileV)

### 